In [15]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [16]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [17]:
subject = 'São Paulo - Consumo de Cimento (t)'
split_index = 191 #Referente aos 230 anos de input  
train_split = split_index + 1 -30

In [18]:
data = pd.read_csv('2003_mo_model_input_SP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH,São Paulo - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,437.972720,0.314974,1.034903e+09,4.730940e+07,23.895559,9.819129e+08,8.310335,0.812723,782.111
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,434.363720,0.317328,1.036092e+09,4.732994e+07,23.901472,9.822666e+08,8.304194,0.812838,789.891
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,433.350868,0.319840,1.037280e+09,4.735047e+07,23.907385,9.826203e+08,8.298053,0.812953,774.309
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,431.825332,0.322104,1.038469e+09,4.737101e+07,23.913298,9.829741e+08,8.291912,0.813068,782.071
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,433.287298,0.324320,1.039658e+09,4.739155e+07,23.919210,9.833278e+08,8.285771,0.813183,841.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,515.306495,0.590649,NaN,NaN,NaN,NaN,NaN,NaN,1112.914
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514.812591,0.588569,NaN,NaN,NaN,NaN,NaN,NaN,1028.144
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,514.856452,0.585196,NaN,NaN,NaN,NaN,NaN,NaN,996.449
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,515.215930,0.581094,NaN,NaN,NaN,NaN,NaN,NaN,1024.272


In [19]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.410736,-1.192538,-1.718526,-0.426010,-1.485366,-2.737181,-0.804344,-2.274413
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.440931,-1.124056,-1.699080,-0.401643,-1.421647,-2.673768,-0.808200,-2.236038
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.449405,-1.050974,-1.679635,-0.377275,-1.357928,-2.610355,-0.812056,-2.197662
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.462169,-0.985091,-1.660189,-0.352908,-1.294210,-2.546942,-0.815911,-2.159286
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.449937,-0.920633,-1.640744,-0.328540,-1.230491,-2.483529,-0.819767,-2.120910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,-1.209684,1.711139,1.107674,-1.587758,-1.453963,0.436685,1.040670,0.784878
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,-1.192369,1.711746,1.098119,-1.580975,-1.459558,0.423023,1.034370,0.748669
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,-1.178463,1.717654,1.088565,-1.574192,-1.465154,0.409361,1.028069,0.712461
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,-1.166182,1.729082,1.079010,-1.567409,-1.470749,0.395699,1.021769,0.676253


In [20]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      706.731
1      697.361
2      859.304
3      762.222
4      763.920
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: São Paulo - Consumo de Cimento (t), Length: 240, dtype: float64

In [21]:
# input para treinamento
train_input = input_data.iloc[:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-1.410736,-1.192538,-1.718526,-0.426010,-1.485366,-2.737181,-0.804344,-2.274413
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-1.440931,-1.124056,-1.699080,-0.401643,-1.421647,-2.673768,-0.808200,-2.236038
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-1.449405,-1.050974,-1.679635,-0.377275,-1.357928,-2.610355,-0.812056,-2.197662
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-1.462169,-0.985091,-1.660189,-0.352908,-1.294210,-2.546942,-0.815911,-2.159286
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-1.449937,-0.920633,-1.640744,-0.328540,-1.230491,-2.483529,-0.819767,-2.120910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,-0.608419,1.023937,1.193677,-1.393041,-1.206846,0.733610,1.413038,1.566441
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,-0.653197,1.033712,1.195577,-1.411276,-1.217300,0.727991,1.392752,1.539489
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,-0.699576,1.049101,1.197477,-1.429511,-1.227755,0.722372,1.372465,1.512536
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,-0.757794,1.059564,1.199378,-1.447746,-1.238210,0.716753,1.352178,1.485583


In [22]:
# Alvo para treinamento
train_target = target_data.iloc[:train_split]
train_target

0      706.731
1      697.361
2      859.304
3      762.222
4      763.920
        ...   
157    757.915
158    923.634
159    756.663
160    872.715
161    844.988
Name: São Paulo - Consumo de Cimento (t), Length: 162, dtype: float64

In [23]:
#input de test (Ano 2021)
test_input = input_data.iloc[train_split:]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),São Paulo - Produção de Cimento (t),São Paulo - value,São Paulo - PIB - Estadual,São Paulo - PIB - Construção Civil,São Paulo - PIB - Per Capita,São Paulo - PIB - Preços de Mercado,São Paulo - Desemprego,São Paulo - IDH
162,-0.601510,-0.653307,0.157879,-1.255326,1.088045,0.763466,-1.213929,-0.820644,1.082634,1.203179,-1.484216,-1.259120,0.705516,1.311604,1.431678
163,-0.786068,-0.653814,0.548855,-1.118679,1.099377,0.752299,-1.292173,-0.853043,1.099539,1.205079,-1.502451,-1.269575,0.699897,1.291318,1.404725
164,-0.830387,-0.645656,0.385685,-0.936336,1.110709,0.741131,-1.324219,-0.879813,1.131286,1.206979,-1.520686,-1.280030,0.694278,1.271031,1.377772
165,-0.801089,-0.634274,0.420965,-0.931790,1.122042,0.729964,-1.344446,-0.906260,1.157706,1.208880,-1.538921,-1.290484,0.688659,1.250744,1.350819
166,-0.959917,-0.624035,0.548870,-1.168522,1.133374,0.718796,-1.381638,-0.932027,1.184668,1.210780,-1.557156,-1.300939,0.683040,1.230457,1.323866
167,-1.022309,-0.619791,0.222499,-1.285611,1.144706,0.707629,-1.411208,-0.955450,1.212391,1.212680,-1.575390,-1.311394,0.677421,1.210170,1.296914
168,-1.074401,-0.620112,-0.614754,-1.446316,1.156038,0.696461,-1.412953,-0.980990,1.240924,1.214581,-1.593625,-1.321849,0.671803,1.189884,1.269961
169,-1.119597,-0.615973,-0.478229,-1.357832,1.167287,0.681823,-1.491464,-1.007316,1.258259,1.211245,-1.597093,-1.329595,0.660179,1.181124,1.250659
170,-1.078648,-0.609777,-0.393157,-1.235682,1.178536,0.667184,-1.573805,-1.034034,1.282135,1.207910,-1.600561,-1.337340,0.648555,1.172365,1.231357
171,-1.055426,-0.602141,-0.557552,-1.065340,1.189784,0.652545,-1.564950,-1.061615,1.306114,1.204574,-1.604029,-1.345086,0.636932,1.163605,1.212054


In [24]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[train_split:split_index + 1]
test_target

162     910.653
163     937.696
164     871.611
165     795.853
166     804.767
167     712.733
168     763.665
169     727.397
170     855.245
171     855.042
172     648.867
173    1003.807
174     855.104
175     913.329
176     849.262
177     897.883
178     860.059
179     771.417
180     860.232
181     773.607
182     790.146
183     886.929
184     941.600
185     817.301
186     957.289
187     978.990
188     906.956
189     980.948
190     872.046
191     692.232
Name: São Paulo - Consumo de Cimento (t), dtype: float64

In [25]:
def validation_splitter(df, div_factor):
    split_factor = len(df)//div_factor
    positions_to_drop = []
    for i in range(split_factor):
        pos = df.shape[0] - (i*6 + 1)
        positions_to_drop.append(pos)
    
    return df.drop(positions_to_drop), df.iloc[positions_to_drop]

In [26]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    train, train_val = validation_splitter(train_input, 6)
    target,target_val = validation_splitter(train_target, 6)
#     display(train)
#     display(train_val)
#     display(target)
#     display(target_val)
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val, target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [27]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(25)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [28]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3105976222, 2805720170, 3391438123, 2518405116, 642653407, 422945028, 3626258037, 45802468, 2817319329, 3422377802, 681545618, 2932850902, 2633540016, 2079934621, 3688915333, 2121287827, 1773717203, 1380785523, 714949175, 164812638, 3212960564, 3131451633, 2974545041, 4142879747, 2061307042]


Step: 0 ___________________________________________
val_loss: 9692.923828125
winner_seed: 3105976222


Step: 1 ___________________________________________
val_loss: 11535.814453125


Step: 2 ___________________________________________
val_loss: 9433.6708984375
winner_seed: 3391438123


Step: 3 ___________________________________________
val_loss: 10289.236328125


Step: 4 ___________________________________________
val_loss: 10716.6689453125


Step: 5 ___________________________________________
val_loss: 9177.4560546875
winner_seed: 422945028


Step: 6 ___________________________________________
val_loss: 10074.7265625


Step: 7 ___________________________________________
val_loss: 9362.62011718

In [29]:
trained_model, history = neural_network_model(train_input, 
                                              train_target, 
                                              want_verbose=1, 
                                              seed=winner_seed)

Epoch 1/10000
5/5 [==============================] - 1s 24ms/step - loss: 1385850.0000 - val_loss: 933882.3750
Epoch 2/10000
5/5 [==============================] - 0s 5ms/step - loss: 931419.1250 - val_loss: 703120.3125
Epoch 3/10000
5/5 [==============================] - 0s 5ms/step - loss: 711080.8750 - val_loss: 550094.5000
Epoch 4/10000
5/5 [==============================] - 0s 5ms/step - loss: 599960.3750 - val_loss: 611562.8125
Epoch 5/10000
5/5 [==============================] - 0s 5ms/step - loss: 592906.5625 - val_loss: 443969.2500
Epoch 6/10000
5/5 [==============================] - 0s 5ms/step - loss: 426762.2500 - val_loss: 412088.8750
Epoch 7/10000
5/5 [==============================] - 0s 5ms/step - loss: 412756.1250 - val_loss: 305358.9062
Epoch 8/10000
5/5 [==============================] - 0s 5ms/step - loss: 364358.6875 - val_loss: 304199.4688
Epoch 9/10000
5/5 [==============================] - 0s 5ms/step - loss: 339644.5312 - val_loss: 317102.1875
Epoch 10/10000
5/

5/5 [==============================] - 0s 6ms/step - loss: 15697.1592 - val_loss: 21014.4824
Epoch 77/10000
5/5 [==============================] - 0s 5ms/step - loss: 15450.7314 - val_loss: 13401.9834
Epoch 78/10000
5/5 [==============================] - 0s 5ms/step - loss: 13372.9609 - val_loss: 19068.2148
Epoch 79/10000
5/5 [==============================] - 0s 5ms/step - loss: 13722.1025 - val_loss: 19065.9648
Epoch 80/10000
5/5 [==============================] - 0s 5ms/step - loss: 15266.7744 - val_loss: 20031.9609
Epoch 81/10000
5/5 [==============================] - 0s 5ms/step - loss: 15259.4863 - val_loss: 17372.3535
Epoch 82/10000
5/5 [==============================] - 0s 5ms/step - loss: 16665.1816 - val_loss: 20487.7109
Epoch 83/10000
5/5 [==============================] - 0s 5ms/step - loss: 12848.0820 - val_loss: 21009.5371
Epoch 84/10000
5/5 [==============================] - 0s 5ms/step - loss: 14403.9531 - val_loss: 22832.1367
Epoch 85/10000
5/5 [=======================

5/5 [==============================] - 0s 6ms/step - loss: 8912.5557 - val_loss: 17489.1992
Epoch 152/10000
5/5 [==============================] - 0s 6ms/step - loss: 11554.2998 - val_loss: 16460.9551
Epoch 153/10000
5/5 [==============================] - 0s 6ms/step - loss: 8923.1582 - val_loss: 14449.6162
Epoch 154/10000
5/5 [==============================] - 0s 6ms/step - loss: 8721.9521 - val_loss: 16672.8145
Epoch 155/10000
5/5 [==============================] - 0s 5ms/step - loss: 11637.2959 - val_loss: 14454.4297
Epoch 156/10000
5/5 [==============================] - 0s 6ms/step - loss: 11727.4248 - val_loss: 20785.7676
Epoch 157/10000
5/5 [==============================] - 0s 6ms/step - loss: 8547.2988 - val_loss: 16267.0264
Epoch 158/10000
5/5 [==============================] - 0s 6ms/step - loss: 9631.4023 - val_loss: 15742.7432
Epoch 159/10000
5/5 [==============================] - 0s 6ms/step - loss: 10383.3115 - val_loss: 15721.4424
Epoch 160/10000
5/5 [===================

Epoch 227/10000
5/5 [==============================] - 0s 6ms/step - loss: 8697.3486 - val_loss: 15798.2119
Epoch 228/10000
5/5 [==============================] - 0s 6ms/step - loss: 8984.7236 - val_loss: 16710.3262
Epoch 229/10000
5/5 [==============================] - 0s 6ms/step - loss: 8852.8086 - val_loss: 17345.9414
Epoch 230/10000
5/5 [==============================] - 0s 6ms/step - loss: 9002.3545 - val_loss: 16013.8857
Epoch 231/10000
5/5 [==============================] - 0s 5ms/step - loss: 9268.9912 - val_loss: 17071.5547
Epoch 232/10000
5/5 [==============================] - 0s 5ms/step - loss: 7901.8926 - val_loss: 18666.7012
Epoch 233/10000
5/5 [==============================] - 0s 5ms/step - loss: 8278.2090 - val_loss: 17394.6094
Epoch 234/10000
5/5 [==============================] - 0s 7ms/step - loss: 8782.3955 - val_loss: 16725.9004
Epoch 235/10000
5/5 [==============================] - 0s 6ms/step - loss: 9036.8076 - val_loss: 16360.5713
Epoch 236/10000
5/5 [=======

5/5 [==============================] - 0s 6ms/step - loss: 9853.8730 - val_loss: 18414.7930
Epoch 303/10000
5/5 [==============================] - 0s 6ms/step - loss: 9422.8135 - val_loss: 19700.4355
Epoch 304/10000
5/5 [==============================] - 0s 6ms/step - loss: 9313.5684 - val_loss: 19502.9277
Epoch 305/10000
5/5 [==============================] - 0s 6ms/step - loss: 9240.2363 - val_loss: 19025.9375
Epoch 306/10000
5/5 [==============================] - 0s 5ms/step - loss: 8870.4404 - val_loss: 20169.7324
Epoch 307/10000
5/5 [==============================] - 0s 5ms/step - loss: 9395.0078 - val_loss: 15268.0400
Epoch 308/10000
5/5 [==============================] - 0s 5ms/step - loss: 9589.7705 - val_loss: 15859.6436
Epoch 309/10000
5/5 [==============================] - 0s 5ms/step - loss: 9128.0400 - val_loss: 14373.4814
Epoch 310/10000
5/5 [==============================] - 0s 5ms/step - loss: 8550.7490 - val_loss: 20078.0039
Epoch 311/10000
5/5 [=======================

...

Epoch 1352/10000
5/5 [==============================] - 0s 5ms/step - loss: 10741.3945 - val_loss: 10579.0371
Epoch 1353/10000
5/5 [==============================] - 0s 6ms/step - loss: 9928.6689 - val_loss: 14264.9229
Epoch 1354/10000
5/5 [==============================] - 0s 6ms/step - loss: 10069.7236 - val_loss: 10721.9805
Epoch 1355/10000
5/5 [==============================] - 0s 6ms/step - loss: 10853.6846 - val_loss: 12030.3428
Epoch 1356/10000
5/5 [==============================] - 0s 5ms/step - loss: 11419.0518 - val_loss: 12370.6064
Epoch 1357/10000
5/5 [==============================] - 0s 5ms/step - loss: 12870.6318 - val_loss: 12377.0088
Epoch 1358/10000
5/5 [==============================] - 0s 5ms/step - loss: 9676.2832 - val_loss: 12006.1982
Epoch 1359/10000
5/5 [==============================] - 0s 5ms/step - loss: 9336.8486 - val_loss: 17067.4043
Epoch 1360/10000
5/5 [==============================] - 0s 5ms/step - loss: 11593.5088 - val_loss: 11712.2383
Epoch 1361/10

5/5 [==============================] - 0s 5ms/step - loss: 8947.9844 - val_loss: 19351.2441
Epoch 1427/10000
5/5 [==============================] - 0s 6ms/step - loss: 10881.7607 - val_loss: 16875.1758
Epoch 1428/10000
5/5 [==============================] - 0s 7ms/step - loss: 10603.1152 - val_loss: 19034.3535
Epoch 1429/10000
5/5 [==============================] - 0s 6ms/step - loss: 11092.2891 - val_loss: 15942.5703
Epoch 1430/10000
5/5 [==============================] - 0s 6ms/step - loss: 9812.4541 - val_loss: 15899.2920
Epoch 1431/10000
5/5 [==============================] - 0s 6ms/step - loss: 8451.9346 - val_loss: 16592.8242
Epoch 1432/10000
5/5 [==============================] - 0s 6ms/step - loss: 9672.4639 - val_loss: 21306.5020
Epoch 1433/10000
5/5 [==============================] - 0s 5ms/step - loss: 9629.2119 - val_loss: 21216.8340
Epoch 1434/10000
5/5 [==============================] - 0s 5ms/step - loss: 10745.7949 - val_loss: 15368.4883
Epoch 1435/10000
5/5 [==========

Epoch 1501/10000
5/5 [==============================] - 0s 6ms/step - loss: 11362.4746 - val_loss: 13152.2246
Epoch 1502/10000
5/5 [==============================] - 0s 6ms/step - loss: 11268.9316 - val_loss: 18240.8359
Epoch 1503/10000
5/5 [==============================] - 0s 5ms/step - loss: 10940.3838 - val_loss: 12423.0488
Epoch 1504/10000
5/5 [==============================] - 0s 6ms/step - loss: 8940.8652 - val_loss: 12276.1260
Epoch 1505/10000
5/5 [==============================] - 0s 5ms/step - loss: 11408.8105 - val_loss: 12475.9521
Epoch 1506/10000
5/5 [==============================] - 0s 5ms/step - loss: 12761.3457 - val_loss: 16734.4004
Epoch 1507/10000
5/5 [==============================] - 0s 6ms/step - loss: 12477.0088 - val_loss: 15936.0117
Epoch 1508/10000
5/5 [==============================] - 0s 5ms/step - loss: 10878.7207 - val_loss: 12835.7607
Epoch 1509/10000
5/5 [==============================] - 0s 5ms/step - loss: 11099.5996 - val_loss: 18283.7051
Epoch 1510/

Epoch 1576/10000
5/5 [==============================] - 0s 5ms/step - loss: 10166.7451 - val_loss: 12267.2012
Epoch 1577/10000
5/5 [==============================] - 0s 6ms/step - loss: 11094.6299 - val_loss: 14915.2422
Epoch 1578/10000
5/5 [==============================] - 0s 6ms/step - loss: 10586.3125 - val_loss: 12737.4980
Epoch 1579/10000
5/5 [==============================] - 0s 5ms/step - loss: 9041.3223 - val_loss: 13178.2588
Epoch 1580/10000
5/5 [==============================] - 0s 6ms/step - loss: 9279.8721 - val_loss: 13203.4697
Epoch 1581/10000
5/5 [==============================] - 0s 6ms/step - loss: 9212.2305 - val_loss: 13140.7939
Epoch 1582/10000
5/5 [==============================] - 0s 5ms/step - loss: 10120.8613 - val_loss: 13489.3135
Epoch 1583/10000
5/5 [==============================] - 0s 5ms/step - loss: 8963.0586 - val_loss: 12976.3057
Epoch 1584/10000
5/5 [==============================] - 0s 5ms/step - loss: 8206.4580 - val_loss: 14151.9600
Epoch 1585/1000

5/5 [==============================] - 0s 5ms/step - loss: 10692.0312 - val_loss: 15919.3477
Epoch 1651/10000
5/5 [==============================] - 0s 6ms/step - loss: 9656.2520 - val_loss: 16635.1426
Epoch 1652/10000
5/5 [==============================] - 0s 5ms/step - loss: 9346.0449 - val_loss: 20474.4473
Epoch 1653/10000
5/5 [==============================] - 0s 5ms/step - loss: 11344.0088 - val_loss: 22372.2129
Epoch 1654/10000
5/5 [==============================] - 0s 5ms/step - loss: 9595.6465 - val_loss: 18851.5039
Epoch 1655/10000
5/5 [==============================] - 0s 6ms/step - loss: 9979.5879 - val_loss: 15732.0742
Epoch 1656/10000
5/5 [==============================] - 0s 7ms/step - loss: 10439.9902 - val_loss: 16013.1797
Epoch 1657/10000
5/5 [==============================] - 0s 6ms/step - loss: 9290.0293 - val_loss: 25541.5957
Epoch 1658/10000
5/5 [==============================] - 0s 5ms/step - loss: 13274.5430 - val_loss: 20826.9902
Epoch 1659/10000
5/5 [==========

In [30]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input.iloc[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [31]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            test_input, 
                                            test_target, 
                                            train_split)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 14ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,940.80011,940.80011,940.80011,975.919922,975.919922,940.80011,940.80011,940.80011,940.80011,940.80011,940.80011,837.918945,940.80011,940.80011,899.820679,899.81958,837.918945,942.229614,940.80011,975.919922,900.270996,899.820679,899.820679,899.820679,899.820679,899.820679,899.820679,894.860718,837.918945,837.918945
Target,910.653,937.696,871.611,795.853,804.767,712.733,763.665,727.397,855.245,855.042,648.867,1003.807,855.104,913.329,849.262,897.883,860.059,771.417,860.232,773.607,790.146,886.929,941.6,817.301,957.289,978.99,906.956,980.948,872.046,692.232
Error,30.147095,3.104126,69.189087,180.066895,171.152893,228.067139,177.135132,213.403137,85.555115,85.758118,291.933105,165.888062,85.696106,27.47113,50.558655,1.936584,22.140076,170.812622,80.568115,202.312927,110.125,12.891663,41.779297,82.519653,57.468323,79.169312,7.135315,86.08728,34.127075,145.686951


In [32]:
display(mae)
display(mape)

99.99619

0.12650836

In [33]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [34]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             train_split)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 13ms/step
Ano-0: |Prediction[[11256.96]] - Target[9887.336000000001]| =  Error: [[1369.624]]; MAPE:[[0.13852306]]
1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[10977.841]] - Target[10216.868999999999]| =  Error: [[760.9717]]; MAPE:[[0.07448188]]
1/1 [==============================] - 0s 14ms/step
Ano-5: |Prediction[[5270.1606]] - Target[5388.461]| =  Error: [[118.30029]]; MAPE:[[0.02195437]]


[array([[1369.624]], dtype=float32),
 array([[760.9717]], dtype=float32),
 array([[118.30029]], dtype=float32)]

749.632

0.07831977